In [1]:
## class
import pandas as pd
import os

class TurtleData:
    """Commom base class for all turtle's data """
    
    C1 = 'Acquisition Time'
    C2 ='Acquisition Start Time'
    C3 ='Iridium CEP Radius'
    C4 ='Iridium Latitude'
    C5 ='Iridium Longitude'
    C6 ='GPS Fix Time'
    C7 ='GPS Fix Attempt'
    C8 ='GPS Latitude'
    C9 ='GPS Longitude'
    C10 ='GPS UTM Zone'
    C11 ='GPS UTM Northing'
    C12 ='GPS UTM Easting'
    C13 ='GPS Altitude'
    C14 ='GPS Horizontal Error'
    C15 ='GPS Horizontal Dilution'
    C16 ='GPS Satellite Bitmap'
    C17 ='GPS Satellite Count'
    C18 ='Underwater Percentage'
    C19 ='Dive Count'
    C20 ='Average Dive Duration'
    C21 ='Dive Duration Standard Deviation'
    C22 ='Maximum Dive Duration'
    C23 ='Maximum Dive Depth'
    C24 ='Duration Limit 1 Dive Count'
    C25 ='Duration Limit 2 Dive Count'
    C26 ='Duration Limit 3 Dive Count'
    C27 ='Duration Limit 4 Dive Count'
    C28 ='Duration Limit 5 Dive Count'
    C29 ='Duration Limit 6 Dive Count'
    C30 ='Layer 1 Percentage'
    C31 ='Layer 2 Percentage'
    C32 ='Layer 3 Percentage'
    C33 ='Layer 4 Percentage'
    C34 ='Layer 5 Percentage'
    C35 ='Layer 6 Percentage'
    C36 ='Layer 7 Percentage'
    C37 ='Layer 8 Percentage'
    C38 ='Layer 9 Percentage'
    C39 ='Layer 10 Percentage'
    C40 ='Layer 1 Dive Count'
    C41 ='Layer 2 Dive Count'
    C42 ='Layer 3 Dive Count'
    C43 ='Layer 4 Dive Count'
    C44 ='Layer 5 Dive Count'
    C45 ='Layer 6 Dive Count'
    C46 ='Layer 7 Dive Count'
    C47 ='Layer 8 Dive Count'
    C48 ='Layer 9 Dive Count'
    C49 ='Layer 10 Dive Count'
    C50 ='Temperature'
    C51 ='Satellite Uplink'
    C52 ='Receive Time'
    C53 ='Repetition Count'
    C54 ='Low Voltage'
    C55 ='Mortality'
    C56 ='Saltwater Failsafe'
    C57 ='Iridium Command'
    C58 ='Schedule Set'
    C59 ='Diagnostic Dive Data'
    C60 ='Predeployment Data'
    C61 ='Error'
    col_names = list([
        C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, 
        C11, C12, C13, C14, C15, C16, C17, C18, C19, C20, 
        C21, C22, C23, C24, C25, C26, C27, C28, C29, C30, 
        C31, C32, C33, C34, C35, C36, C37, C38, C39, C40, 
        C41, C42, C43, C44, C45, C46, C47, C48, C49, C50, 
        C51, C52, C53, C54, C55, C56, C57, C58, C59, C60, 
        C61
    ])
    gps_col_names = list([
        C1, C2, C6, C7, C8, C9
    ])	
    
    def __init__(self, tag):
        self.turtleTag = tag
        self.df = pd.DataFrame()
        self.gpsDf = pd.DataFrame()
    #def addElement(self, row, header):
        #self.__dict__= dict(zip(header, row))

    def addDataFromCsv(self, filename):
        temporaryDf = pd.read_csv(filename, skiprows=23, names=TurtleData.col_names)
        
        #print(f' ITS CURRENT DF IS: {self.df}') 
        #print('--------------')
        #print(f' ITS TEMPORARY DF IS {temporaryDf}') 
        
        self.df = self.df.append(temporaryDf, ignore_index=True)
        self.df.sort_values("Acquisition Time", inplace = True)

    def getTag(self):
        return self.turtleTag

    def getDf(self):        
        return self.df
    
    def giveAllGpsDf(self):
        # see all the columns in the df
        #print(self.df.columns)
        # see one column at a time        
        self.gpsDf = self.df.copy()
        print(TurtleData.gps_col_names)
        tempList = TurtleData.gps_col_names.copy()
        for c in self.gpsDf.columns:
            print(c)
            if c not in tempList:
                self.gpsDf.drop(c, inplace=True, axis=1)
            else:
                tempList.remove(c)
                
        if tempList:
            print("Colummn Data missing in!")
        else:
            print("The dataframe contains all the GPS columns")
        
        print('-----DF with NaN values ---------')
        print(self.gpsDf)
        
        #### Eliminate those GPS's null (NaN) rows from the dataframe
        self.gpsDf.drop(self.gpsDf[~self.gpsDf['GPS Latitude'].notna()].index, inplace=True)
        self.gpsDf.reset_index(drop=True, inplace=True) # reset index
        
        print('-----SAME DF without NaN values ---------')
        print(self.gpsDf)
        print('--------------')
        
        ####Create a column for id GPS points to the left
        trackId = self.gpsDf.index + 1
        self.gpsDf.insert(0, 'Track ID', trackId)
        
        print(self.gpsDf)

In [2]:
# functions
## To run with Debug:
#DIRNAME = os.path.dirname(__file__)
#ASSETS_FOLDER = os.path.join(DIRNAME, 'assets')
##ASSETS_FOLDER_OBJ = "data_analysis\\assets"
#ASSETS_FOLDER_ITENS = os.listdir(ASSETS_FOLDER)# ("data_analysis/assets")

# To run with terminal:
ASSETS_FOLDER = "assets"
ASSETS_FOLDER_ITENS = os.listdir(ASSETS_FOLDER)# ("assets")

TAG_TURTLE_1 = '710333A'
TAG_TURTLE_2 = '710348A'

INITIAL_TAG_DIGITS = '7103'

# Replace spaces in filenames with underlines
def replace_space_with_underline(file_name):
    return file_name.replace(" ", "_")

# Convert excel files into csv
def converting_excel_file_into_csv_file(folder_obj, file):        
    # read excel   
    df_xlsx = pd.read_excel(os.path.join(folder_obj, file))
    # change file format
    file_in_csv = file.replace(".xlsx", ".csv")
    # transform excel to csv file with path to store the CSV file
    df_xlsx.to_csv(os.path.join(folder_obj, file_in_csv), index = False)        

# Check if some excel file has not been converted into csv yet
def check_for_excel_files():
    all_my_files = []
    n = 0
    for file in ASSETS_FOLDER_ITENS:
        # put all the file names in the same format
        file = replace_space_with_underline(file).lower()
        all_my_files.append(file)
    
    # Create a copy of list
    for file in all_my_files[:]:
        if file.endswith('.xlsx'):
            print('- Excel file = ' + file)
            file_name = file.split('.', 1)[0] # remove everything (the format) after the dot
            # remove the excel file from my all_my_files list
            all_my_files.remove(file)            
            # check if another file with the same name in the folder exists
            if any(file_name in word for word in all_my_files):            
                print(f"-- Excellent! We've already converted the excel file \'{file_name}\' into csv file")
            else:
                print(f'-- Oh No! The excel file \'{file_name}\' has been not converted. Converting it into csv file...')
                # Call function "Convert excel files into csv"
                converting_excel_file_into_csv_file(ASSETS_FOLDER_OBJ, file)
                file_in_csv = file.replace(".xlsx", ".csv") 
                all_my_files.append(file_in_csv)
                print('---> ' + file_in_csv + ' has been created!')
                
    # Updated all_my_files List
    print('--- CSV files in the assets folder: ', all_my_files)

def getTurtlesData():
    split_char = '_'
    csvs = []        
    turtlesData = []
    #turtleDfs = []
    for file in ASSETS_FOLDER_ITENS:
        if file.endswith('.csv'):
            # put all the file names in the same format
            csv_string_filename = replace_space_with_underline(file).lower()
            filename_splitted = csv_string_filename.split(split_char)                        
            for word in filename_splitted:
                if word.startswith(INITIAL_TAG_DIGITS):
                    csvs.append(file)
                    currentFileCsv = ASSETS_FOLDER + '\\' + file
                    print('--------------')
                    print("Found TAG ("+ word +") in filename , check if tag is already associated with an object...")

                    #--------------------
                                
                    foundTurtleData = None
                    # check inside the list if the turtle has already been created with that tag (word)
                    for obj in turtlesData:
                        if obj.getTag() == word:
                            foundTurtleData = obj
                            break    
                    #--------------------    
                                    
                    if foundTurtleData == None:
                        print("Instance for TAG ("+ word +") NOT found! Creating a new instance...")
                        # create a TurtleData obj with the turtle tag
                        foundTurtleData = TurtleData(word)
                        turtlesData.append(foundTurtleData)
                        print("Instance for TAG ("+ word +") CREATED!")
                    else:
                        print("Instance for TAG ("+ word +") ALREADY EXISTS, skipping object creation!")
                        print('--------------')

                    # for the instances turtleData objs in the list (for each turtle tag):
                    foundTurtleData.addDataFromCsv(currentFileCsv)                    

    return turtlesData

def checkInstancesAndItsDfs(turtlesData):
    print('Created instances for Obj turtleData: ')
    for turtleData in turtlesData:        
        print(turtleData.getTag())
    print('--------------')
    print('Created Dataframes: ')
    i = 0
    for turtleData in turtlesData: 
        print(f'turtlesData[{i}].df')
        print(turtleData.turtleTag)
        print(turtleData.df)
        print('--------------')
        i+=1

In [3]:
# main

# Check if some excel file has not been converted into csv yet
check_for_excel_files()
turtlesData = getTurtlesData()

# see instances for Obj turtleData created and its dataframes
checkInstancesAndItsDfs(turtlesData)  

- Excel file = mytest.xlsx
-- Excellent! We've already converted the excel file 'mytest' into csv file
- Excel file = tag_710333a_20_sept.xlsx
-- Excellent! We've already converted the excel file 'tag_710333a_20_sept' into csv file
--- CSV files in the assets folder:  ['710333a_93_condensed.csv', '710348a_49_condensed.csv', 'mytest.csv', 'tag_710333a_20_sept.csv']
--------------
Found TAG (710333a) in filename , check if tag is already associated with an object...
Instance for TAG (710333a) NOT found! Creating a new instance...
Instance for TAG (710333a) CREATED!
--------------
Found TAG (710348a) in filename , check if tag is already associated with an object...
Instance for TAG (710348a) NOT found! Creating a new instance...
Instance for TAG (710348a) CREATED!
--------------
Found TAG (710333a) in filename , check if tag is already associated with an object...
Instance for TAG (710333a) ALREADY EXISTS, skipping object creation!
--------------
Created instances for Obj turtleData: 
71

In [4]:
def getGpsDataframes(turtlesData):
    for turtleData in turtlesData:
        turtleData.giveAllGpsDf()

In [5]:
getGpsDataframes(turtlesData)

['Acquisition Time', 'Acquisition Start Time', 'GPS Fix Time', 'GPS Fix Attempt', 'GPS Latitude', 'GPS Longitude']
Acquisition Time
Acquisition Start Time
Iridium CEP Radius
Iridium Latitude
Iridium Longitude
GPS Fix Time
GPS Fix Attempt
GPS Latitude
GPS Longitude
GPS UTM Zone
GPS UTM Northing
GPS UTM Easting
GPS Altitude
GPS Horizontal Error
GPS Horizontal Dilution
GPS Satellite Bitmap
GPS Satellite Count
Underwater Percentage
Dive Count
Average Dive Duration
Dive Duration Standard Deviation
Maximum Dive Duration
Maximum Dive Depth
Duration Limit 1 Dive Count
Duration Limit 2 Dive Count
Duration Limit 3 Dive Count
Duration Limit 4 Dive Count
Duration Limit 5 Dive Count
Duration Limit 6 Dive Count
Layer 1 Percentage
Layer 2 Percentage
Layer 3 Percentage
Layer 4 Percentage
Layer 5 Percentage
Layer 6 Percentage
Layer 7 Percentage
Layer 8 Percentage
Layer 9 Percentage
Layer 10 Percentage
Layer 1 Dive Count
Layer 2 Dive Count
Layer 3 Dive Count
Layer 4 Dive Count
Layer 5 Dive Count
Layer 6

### END OF ALL GPS DF, TO SEE RELIBLE GPS GO TO scripts_rewrite_proj1_jupyter_3